In [ ]:
import sys
from google.colab import drive
drive.mount('/content/drive')
sys.path.append(r"/content/drive/MyDrive/")

In [1]:
import config
from loader_util.preprocessing import (
    ImageToArrayPreprocessor,
    SimplePreprocessor,
    PatchPreprocessor,
    MeanPreprocessor
)

from loader_util.callbacks import TrainingMonitor
from loader_util.io import HDF5DatasetGenerator
from loader_util.nn.conv import AlexNet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import json
import os

In [3]:
# load the dataset means
mean_filepath = r"C:\GoogleDrive\GitHub\deep_learning\misc\courses\pyimage\practitioner-bundle\chap9_cats_dogs_dataprep\output\kaggle_cats_dogs.json"

train_path = r""
valid_path = r""
test_path = r""
callback_filepath = r""
model_save_filepath = r""

# augmentor
aug = ImageDataGenerator(rotation_range=20,
                         zoom_range=0.15,
                         width_shift_range=0.2,
                         height_shift_range=0.2,
                         shear_range=0.15,
                         horizontal_flip=True,
                         fill_mode='nearest')

means = json.loads(open(mean_filepath).read())

# init the preprocessors
sp = SimplePreprocessor(227, 227)
pp = PatchPreprocessor(227, 227)
mp = MeanPreprocessor(rmean=means["r"],
                      gmean=means["g"],
                      bmean=means["b"])
iap = ImageToArrayPreprocessor()


In [ ]:
# init the train and valid data gens
BATCH_SIZE = 128
train_gen = HDF5DatasetGenerator(dbpath=train_path,
                                 batch_size=BATCH_SIZE,
                                 aug=aug,
                                 preprocessors=[pp, mp, iap])
valid_gen = HDF5DatasetGenerator(dbpath=valid_path,
                                 batch_size=BATCH_SIZE,
                                 preprocessors=[sp, mp, iap])

In [ ]:
# model compilation
print(f"[INFO] compiling the model......")
opt = Adam(learning_rate=0.001)
model = AlexNet.build(height=227, width=227, depth=3, classes=2, reg=0.0002)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
callbacks = [TrainingMonitor(figPath=callback_filepath)]


In [ ]:
# train the network
H = model.fit(
    train_gen.generator(),
    steps_per_epoch=train_gen.num_images // BATCH_SIZE,
    validation_data=valid_gen.generator(),
    validation_steps=valid_gen.num_images // BATCH_SIZE,
    epochs=75,
    callbacks=callbacks,
    max_queue_size=10,
    verbose=1
)

In [ ]:
# save the model
print(f"[INFO] serialising the model......")
model.save(model_save_filepath, overwrite=True)
train_gen.close()
valid_gen.close()